# Text Classification 
# ANSHU PANDEY
# PHONE NO: +91 9560547006

In [ ]:
!pip install transformers
!pip3 install tensorflow_text>=2.0.0rc0
!pip install tqdm
!pip install np_utils

In [162]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow_hub as hub
import spacy
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [163]:
categories=pd.read_excel('categories.xlsx')
description=pd.read_excel('sample.xlsx')

In [164]:
categories.head(5)

,catID,catName,catEnabled,catParentID,catLevelNum,catOpenItemCnt,catFullName,catParentList,catSubcatCnt,catTopCatID,catNewItemCnt,catEndTodayItemCnt,catGoingItemCnt,catTaxCode,catNameOrder,AllowMultiselectInSearch,TaxProductCategory
0,1,Antiques,1,-1,1,542,Antiques,1|Antiques,0,1,2,58,0,P9999999,NaN,False,GTPP
1,2,Books/Movies/Music,1,-1,1,17553,Books/Movies/Music,2|Books/Movies/Music,3,2,67,8523,30,PB100000,NaN,False,GPUB
2,3,Coins & Stamps,0,-1,1,0,Coins & Stamps,3|Coins & Stamps,0,3,0,0,0,P9999999,NaN,False,NaN
3,4,Collectibles,1,-1,1,23892,Collectibles,4|Collectibles,33,4,151,4113,94,P9999999,NaN,False,COLL
4,5,Computers,0,-1,1,0,Computers,5|Computers,0,5,0,0,0,P9999999,NaN,False,NaN


In [165]:
categories.columns

Index(['catID', 'catName', 'catEnabled', 'catParentID', 'catLevelNum',
       'catOpenItemCnt', 'catFullName', 'catParentList', 'catSubcatCnt',
       'catTopCatID', 'catNewItemCnt', 'catEndTodayItemCnt', 'catGoingItemCnt',
       'catTaxCode', 'catNameOrder', 'AllowMultiselectInSearch',
       'TaxProductCategory'],
      dtype='object')

In [166]:
 description.head()

,item Title,catID
0,1847 Rogers Bros Silverplate 'Tipped' Flatware,1
1,"Vintage Or Antique? 30"" Wire Rug Beater",1
2,coca cola vintage crate/flat,1
3,2 Pc Buck Pottery,1
4,Antique Ornate Door Plate & Handle,1


In [167]:
# dropping columns from [2:] in categories as there is no use of these columns 
col=categories.columns[2:17]
categories.drop(col,inplace=True,axis=1)


In [263]:
categories.head(4)

,catID,catName
0,1,Antiques
1,2,Books/Movies/Music
2,3,Coins & Stamps
3,4,Collectibles


In [169]:
stop.append('or')

In [170]:
# removing puncutation
description['item Title'] = description['item Title'].str.replace('[^\w\s]','')
# converting to str
description['item Title']=description['item Title'].astype(str)
description['item Title']=description['item Title'].apply(lambda x:x.lower())
#lemmitization
description['item Title'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
# word tokenize
description['item Title']=description['item Title'].apply(lambda x:word_tokenize(x))
# stop word removal
description['item Title']=description['item Title'].apply(lambda x: [item for item in x if item not in stop])


In [171]:
description

,item Title,catID
0,"[1847, rogers, bros, silverplate, tipped, flat...",1
1,"[vintage, antique, 30, wire, rug, beater]",1
2,"[coca, cola, vintage, crateflat]",1
3,"[2, pc, buck, pottery]",1
4,"[antique, ornate, door, plate, handle]",1
...,...,...
9995,"[decorative, carved, wooden, bowl]",2264
9996,"[467lbs, commemorative, history, coins]",2269
9997,"[graduation, year, 1996, us, mint, proof, set]",2269
9998,"[1889, morgan, silver, dollar, us, liberty, 1,...",2269


In [174]:
description['item Title']=description['item Title'].apply(lambda row: " ".join([w for w in row]))

In [179]:
save_df=description

In [238]:
save_df.head(5)

,item Title,catID
0,1847 rogers bros silverplate tipped flatware,1
1,vintage antique 30 wire rug beater,1
2,coca cola vintage crateflat,1
3,2 pc buck pottery,1
4,antique ornate door plate handle,1


In [280]:
from sklearn.feature_extraction.text import TfidfVectorizer
#sentences_train, sentences_valid, y_train, y_valid

tfid_vectorizer = TfidfVectorizer("english")
tfid_matrix = tfid_vectorizer.fit_transform(save_df['item Title'])
# collect the tfid matrix in numpy array
array = tfid_matrix.todense()
df = pd.DataFrame(array)
df['catID']=save_df['catID']
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,12601,12602,12603,12604,12605,12606,12607,12608,12609,12610,12611,12612,12613,12614,12615,12616,12617,12618,12619,12620,12621,12622,12623,12624,12625,12626,12627,12628,12629,12630,12631,12632,12633,12634,12635,12636,12637,12638,12639,catID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


Multinomial Naive Bayes

In [281]:
X=df.drop(['catID'],axis=1)
Y=df['catID']
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.25, random_state=1000)

In [282]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train,y_train )


In [283]:


y_pred = clf.predict(X_valid)
# Evaluation
acc = accuracy_score(y_valid, y_pred)

print("Accuracy on the dataset: {:.2f}".format(acc*100))

Accuracy on the dataset: 16.16


RNN+LSTM

In [244]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(save_df['item Title'])
sequences = tok.texts_to_sequences(save_df['item Title'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [245]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [246]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 96,337
Trainable params: 96,337
Non-trainable params: 0
_________________________________________________________________


In [247]:
model.fit(sequences_matrix,save_df['catID'],batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
22/63 [=========>....................] - ETA: 16s - loss: -3785.3650 - accuracy: 0.0039ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4483, in <genexpr>
    ta.write(time, out) for ta, out in zip(output_ta_t, flat_output))  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 249, in wrapped
    error_in_function=error_in_function)
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4483, in <genexpr>
    ta.write(time, out) for 

Accuracy is too low

# Linear SVM

In [277]:
# Training
model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

# Evaluation
acc = accuracy_score(y_valid, y_pred)

print("Accuracy of validation dataset: {:.2f}".format(acc*100))

Accuracy of validation dataset: 43.24


SO, we are getting highest accuracy using Linear SVM

In [285]:
test=pd.read_excel('text.xlsx')

In [286]:
# removing puncutation
test['text'] = test['text'].str.replace('[^\w\s]','')
# converting to str
test['text']=test['text'].astype(str)
test['text']=test['text'].apply(lambda x:x.lower())
#lemmitization
test['text'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
# word tokenize
test['text']=test['text'].apply(lambda x:word_tokenize(x))
# stop word removal
test['text']=test['text'].apply(lambda x: [item for item in x if item not in stop])


In [291]:
test['text']=test['text'].apply(lambda row: " ".join([w for w in row]))
test

,text
0,happy potter book
1,diamond necklaces
2,football final match
3,jeans tshirt long pants
4,educational books


In [265]:
test_features = vectorizer.transform(test)
   
y_pred = model.predict(test_features)

In [292]:

tfid_matrix = tfid_vectorizer.transform(test['text'])
# collect the tfid matrix in numpy array
array = tfid_matrix.todense()
test_df = pd.DataFrame(array)
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,12600,12601,12602,12603,12604,12605,12606,12607,12608,12609,12610,12611,12612,12613,12614,12615,12616,12617,12618,12619,12620,12621,12622,12623,12624,12625,12626,12627,12628,12629,12630,12631,12632,12633,12634,12635,12636,12637,12638,12639
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [293]:
y_pred = model.predict(test_df)

In [301]:
test['catID']=y_pred

In [304]:
test

,text,catID
0,happy potter book,99
1,diamond necklaces,86
2,football final match,411
3,jeans tshirt long pants,155
4,educational books,128


In [305]:
df_inner = pd.merge(test,categories, on='catID', how='inner')

df_inner

,text,catID,catName
0,happy potter book,99,Books
1,diamond necklaces,86,Necklaces
2,football final match,411,Football
3,jeans tshirt long pants,155,Pants
4,educational books,128,Children's
